# generate data

In [1]:
import generate_data
train,validation,test,cfg= generate_data.generate_data(0)

(79000, 5, 2)
(20000, 5, 2)
(1000, 5, 2)


In [2]:
import torch
device ="cpu"
#print(device)

In [3]:
from torch.utils.data import DataLoader
import utils_data as ud
test_set = ud.Dataset(test[0],test[1],test[2])
#testloader = DataLoader(test_set, batch_size=config.batch_size, drop_last = True, shuffle=True, pin_memory=True)

# load model parameters

In [4]:
import sys
sys.path.append('../LRP')

In [5]:
import utils_model as um

loading_path='model/best_checkpoint1'
wi_tuples,wh_tuples,bi_tuples,bh_tuples,wl,bl= um.model_parameters(loading_path,device,cfg)

## load model

In [6]:
model=um.load_model(cfg,device,loading_path)

In [7]:
model

EncoderDecoder(
  (encoder): RNNEncoder(
    (rnn_layer): GRU(2, 128, num_layers=2, batch_first=True)
  )
  (decoder_cell): DecoderCell(
    (decoder_rnn_cell): GRU(3, 128, num_layers=2, batch_first=True)
    (out1): Linear(in_features=128, out_features=128, bias=True)
    (relu): ReLU()
    (out2): Linear(in_features=128, out_features=1, bias=False)
    (dropout): Dropout(p=0, inplace=False)
  )
)

# test point

In [8]:
import numpy as np

tpoint=test_set[14]

# tensors
inputp,inputf,labels=torch.tensor(np.expand_dims(tpoint[0],axis=0)),torch.tensor(np.expand_dims(tpoint[1],axis=0)),torch.tensor(np.expand_dims(tpoint[2],axis=0))
inputp,inputf,labels = inputp.double(), inputf.double(),labels.double()
# Model computations
prediction =model.forward(inputp, inputf)
print(labels)
print('.....')
print(prediction)

tensor([[[3.5000],
         [3.5000],
         [2.5000]]], dtype=torch.float64)
.....
tensor([[[3.5109],
         [3.5185],
         [2.4903]]], dtype=torch.float64, grad_fn=<CopySlices>)


# lrp

In [9]:
import lrp_function as lrp

len_static=cfg.static_feat_len
stat=[]
mask=(0,2) ## mask
lrp_seq2seq= lrp.lrp_decoder_encoder(cfg,len_static, wi_tuples,wh_tuples,bi_tuples,bh_tuples,
                                          wl,bl)

## y: output of forward pass LRP
## Rxf: relevance scores of input sequence forecast
## Rx : relevance scores of input sequence historical
## Ry: relevance scores of intermidiate decoder outputs
## Rstatic: relevance scores of static input (satation number) 
y,Rxf,Rx,Ry,Rstatic=lrp_seq2seq.lrp_seq2seq(inputp,inputf,stat,mask,epsilon=0.001,bias_factor=0.0) #set bias_factor to 1 for sanity check

In [10]:
print(Rx)

[[ 0.51249262  1.01607014]
 [-0.50898511 -0.48943871]
 [-0.4827934   0.95262141]
 [-0.49999796  0.        ]
 [ 1.01315237  0.        ]]


In [11]:
print(inputp)

tensor([[[ 1.,  2.],
         [-1., -1.],
         [-1.,  2.],
         [-1.,  0.],
         [ 2.,  0.]]], dtype=torch.float64)


In [12]:
Rxf

array([[ 0.        ,  2.06481566],
       [ 0.95854343, -0.96348574],
       [-0.99003967,  0.        ]], dtype=float128)

In [13]:
inputf

tensor([[[ 0.,  2.],
         [ 1., -1.],
         [-1.,  0.]]], dtype=torch.float64)

# Input x Graient

In [16]:
import sys
sys.path.append('/home/fe/mirzavandborujeni/anaconda3/envs/myenv/lib/python3.8/site-packages')

In [17]:
from captum.attr import IntegratedGradients,Saliency,InputXGradient

In [18]:
input_x_gradient = InputXGradient(model)
heatmap = input_x_gradient.attribute((inputp,inputf),target=mask[1])

/home/fe/mirzavandborujeni/anaconda3/envs/myenv/lib/python3.8/site-packages/captum/_utils/gradient.py:56: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/home/fe/mirzavandborujeni/anaconda3/envs/myenv/lib/python3.8/site-packages/captum/_utils/gradient.py:56: UserWarning: Input Tensor 1 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


In [19]:
inpg_p=heatmap[0]
inpg_f=heatmap[1]

In [20]:
inpg_p

tensor([[[ 0.4931,  0.9828],
         [-0.4928, -0.4905],
         [-0.4986,  1.0022],
         [-0.4979,  0.0000],
         [ 1.0030,  0.0000]]], dtype=torch.float64, grad_fn=<MulBackward0>)

In [21]:
inpg_f

tensor([[[ 0.0000,  2.0268],
         [ 1.0075, -1.0134],
         [-1.0315,  0.0000]]], dtype=torch.float64, grad_fn=<MulBackward0>)

# Integrated Gradients 

In [22]:
# applying integrated gradients on the model with the same data point
ig = IntegratedGradients(model)
heatmap1  = ig.attribute((inputp,inputf),target=mask[1],n_steps=30)

In [23]:
intg_p=heatmap1[0]
intg_f=heatmap1[1]

In [24]:
intg_p

tensor([[[ 0.4948,  0.9873],
         [-0.4951, -0.4927],
         [-0.4999,  1.0023],
         [-0.5015,  0.0000],
         [ 1.0051,  0.0000]]], dtype=torch.float64)

In [25]:
intg_f

tensor([[[ 0.0000,  2.0287],
         [ 1.0140, -1.0208],
         [-1.0368,  0.0000]]], dtype=torch.float64)

# Saliency

In [26]:
sal=Saliency(model)
heatmap2  = sal.attribute((inputp,inputf),target=mask[1],abs=False)

In [27]:
sa_p=heatmap2[0]
sa_f=heatmap2[1]

In [28]:
sa_p

tensor([[[0.4931, 0.4914],
         [0.4928, 0.4905],
         [0.4986, 0.5011],
         [0.4979, 0.4972],
         [0.5015, 0.4999]]], dtype=torch.float64)

In [29]:
sa_f

tensor([[[1.0016, 1.0134],
         [1.0075, 1.0134],
         [1.0315, 1.0363]]], dtype=torch.float64)